In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout, Conv1D, MaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential, load_model
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
q = pd.read_csv("Questions.csv", encoding='ISO-8859-1')
t = pd.read_csv("Tags.csv", encoding='ISO-8859-1')
q = q[:1000]

In [3]:
q.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


In [4]:
t.head()

,Id,Tag
0,80,flex
1,80,actionscript-3
2,80,air
3,90,svn
4,90,tortoisesvn


In [5]:
q = q[["Id","Title","Body"]]
q.head()

,Id,Title,Body
0,80,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


In [6]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    return re.sub(cleanr, '', raw_html)

In [7]:
q['Body'] = q['Body'].apply(cleanhtml)
q["Title"] = q["Title"].map(str) + " " + q["Body"]
q = q[["Id","Title"]]
q["Title"] = q["Title"].apply(lambda x: re.sub('[^0-9a-zA-Z ]+', '', x.lower()))
q.head()

,Id,Title
0,80,sqlstatementexecute multiple queries in one s...
1,90,good branching and merging tutorials for torto...
2,120,aspnet site maps has anyone got experience cre...
3,180,function for creating color wheels this is som...
4,260,adding scripting functionality to net applicat...


In [8]:
def getKeys(index):
    tags = t[t["Id"] == index]
    myTags = list(tags["Tag"])
    out = ""
    #for x in myTags:
    #    out+= str(x)+","
    #return out[0:len(out)-1]
    return myTags[0]

In [9]:
q["Tag"] = q['Id'].apply(getKeys)
q = q.set_index('Id')
q.head()

,Title,Tag
Id,,
80,sqlstatementexecute multiple queries in one s...,flex
90,good branching and merging tutorials for torto...,svn
120,aspnet site maps has anyone got experience cre...,sql
180,function for creating color wheels this is som...,algorithm
260,adding scripting functionality to net applicat...,c#


In [10]:
print(len(q["Title"]))
print(len(q["Tag"]))

1000
1000


In [11]:
tokenizer = Tokenizer(num_words = 1000, split=" ")
tokenizer.fit_on_texts(q["Title"].values)
X = tokenizer.texts_to_sequences(q["Title"].values)

In [12]:
X = pad_sequences(X, maxlen=1000)

In [13]:
print(len(X[0]))

1000


In [14]:
total_words = q.count()[0]
print(total_words)
model = Sequential()
model.add(Embedding(total_words, 10, input_length = 1000))

1000


In [15]:
model.add(LSTM(100))
model.add(Dense(1000, activation='softmax'))

In [16]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1000, 10)          10000     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               44400     
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              101000    
Total params: 155,400
Trainable params: 155,400
Non-trainable params: 0
_________________________________________________________________


In [17]:
tokenizer.fit_on_texts(q["Tag"].values)
Y = tokenizer.texts_to_sequences(q["Tag"].values)
Y = pad_sequences(Y, maxlen=1000)
#Y = pd.get_dummies(q["Tag"]).values
print(len(X))
print(len(Y))
print(Y)
print(len(q.index))

1000
1000
[[  0   0   0 ...   0   0 416]
 [  0   0   0 ...   0   0 360]
 [  0   0   0 ...   0   0  61]
 ...
 [  0   0   0 ...   0  61  46]
 [  0   0   0 ...   0   0  34]
 [  0   0   0 ...   0 209  48]]
1000


In [18]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size=0.2,random_state = 42)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(800, 1000) (800, 1000)
(200, 1000) (200, 1000)


In [19]:
model.fit(X, Y, epochs=10, batch_size=32, validation_data=(x_test, y_test))

Train on 1000 samples, validate on 200 samples
Epoch 1/10
1000/1000 [==============================] - 31s 31ms/step - loss: 945.0999 - acc: 0.7440 - val_loss: 129.4228 - val_acc: 0.7700
Epoch 2/10
1000/1000 [==============================] - 29s 29ms/step - loss: 130.5712 - acc: 0.7700 - val_loss: 95.6308 - val_acc: 0.7700
Epoch 3/10
1000/1000 [==============================] - 28s 28ms/step - loss: 116.9847 - acc: 0.7700 - val_loss: 92.8563 - val_acc: 0.7700
Epoch 4/10
1000/1000 [==============================] - 28s 28ms/step - loss: 115.6608 - acc: 0.7700 - val_loss: 91.6649 - val_acc: 0.7700
Epoch 5/10
1000/1000 [==============================] - 29s 29ms/step - loss: 115.2674 - acc: 0.7700 - val_loss: 90.6223 - val_acc: 0.7700
Epoch 6/10
1000/1000 [==============================] - 33s 33ms/step - loss: 114.8132 - acc: 0.7700 - val_loss: 91.2313 - val_acc: 0.7700
Epoch 7/10
1000/1000 [==============================] - 32s 32ms/step - loss: 115.1305 - acc: 0.7700 - val_loss: 94.55

In [ ]:
print(model.predict)

In [20]:
model.save("qTot.hdf5")